## Relevant Links
- Examples: https://docs.google.com/spreadsheets/d/14pdiODt9PmsD2F___Lpv2rO-RByTtC8PYcEfxL8iIuY/edit#gid=186467173
- LangChain: https://github.com/hwchase17/langchain
- Chroma: no docker or API_KEY - https://www.trychroma.com/
- OpenAI account usage: https://platform.openai.com/account/usage

## Tasks

- **Eric & Jacob**: Validation (`validate_stages.py` is not yet working, and has not been incorporated)
- **Jacob**: Label class recognition
    - determine what classes the user is referring to for each label field
- **Allen**: Evaluation Identification Stage:
    - `links/evaluation_run_selector.py` 
    - add stage to `dataset_view_generator.py`
    - add `prompts/evaluation_task_rules.txt` prompt
    - add examples to Examples spreadsheet in new tab, and then put in `examples` folder
- **Leila**: Add support for `hardness` brain runs
    - fill out the template in `links/brain_run_selector.py` 
    - add `prompts/hardness_task_rules.txt` prompt modeled after uniqueness and mistakenness.
    - new examples tab `hardness` modeled after `uniqueness` and `mistakenness` tabs and then put in `examples` folder
- **Vini**: More examples
    - scrape examples from community Slack covering as wide a range of scenarios as possible
    - complex filters and view expressions!
    - videos
    - every type of label
    - varied naming conventions

## Additional notes

- Want to validate embedded fields
- At present, there is no support for multi-line Python code to generate views. It all needs to be done inline
- No support for groups, 3D/point clouds, `concat()`, or `mongo()`.

## Getting started

`pip install openai langchain chroma`

Then create an OpenAI account and generate an API key

`export OPENAI_API_KEY=...`

In [2]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

## Get similar prompts

In [3]:
from links.view_stage_example_selector import generate_view_stage_examples_prompt

In [4]:
query = "Five random images from the dataset"

In [5]:
view_stage_examples_prompt = generate_view_stage_examples_prompt(dataset, query)

Using embedded DuckDB without persistence: data will be transient


In [6]:
print(view_stage_examples_prompt)

Generate code to produce the FiftyOne view stages for the following prompt:


Input: show me 50 random samples
Output: dataset.take(50)

Input: Take two random samples from the dataset
Output: dataset.take(2)

Input: 10 most unique images
Output: dataset.filter_labels("predictions", F("confidence") > 0.95)

Input: 10 random images with tables
Output: dataset.match(
    F("ground_truth.detections.label").contains("table")
).take(10)

Input: show me random samples
Output: dataset.shuffle()

Input: skip the first 10 samples
Output: dataset.skip(10)

Input: first 100 samples
Output: dataset.limit(100)

Input: Don't show me the fifth and sixth samples
Output: ids = [dataset.skip(4).first().id, dataset.skip(4).first().id];
dataset.exclude(ids)

Input: Images that only contain dogs
Output: dataset.match(
    F("ground_truth.detections.label").is_subset(["dog"])
)

Input: ten least wrong predictions
Output: dataset.sort_by("mistakenness", reverse=False)[:10]

Input: Five random images from the

## Generate View Stage Descriptions Prompt

In [7]:
from links.view_stage_description_selector import generate_view_stage_descriptions_prompt

In [8]:
view_stage_descriptions_prompt = generate_view_stage_descriptions_prompt(view_stage_examples_prompt)

In [9]:
print(view_stage_descriptions_prompt)

Here is some information about view stages you might want to use:


    View stage: skip
    Description: Omits the given number of samples from the head of the collection. Non-positive skip values result in no samples being omitted
    Inputs: skip: int

    

    View stage: take
    Description: Randomly samples the given number of samples from the collection
    Inputs: size: int (non-positive values result in an empty view), seed: optional random seed to use for sample selection

    

    View stage: match
    Description: Filters the samples in the collection by the given filter
    Inputs: filter: ViewExpression or MongoDB expression that returns a boolean describing the filter to apply

    

    View stage: exclude
    Description: Exclude samples with specified IDs
    Inputs: sample_ids: IDs to exclude. Can be single ID, iterable of IDs, Sample or SampleView instances, iterable of Sample or SampleView instances, or SampleCollection

    

    View stage: filter_labels
    D

## Brain method selector

In [7]:
from links.brain_method_selector import select_brain_methods

In [8]:
query = "Five most unique images from the dataset"
select_brain_methods(query)

['uniqueness']

In [127]:
query = "50 least unique images from the dataset that were hard"
select_brain_methods(query)

['uniqueness', 'hardness']

In [9]:
query = "Five most mistaken images from the dataset"
select_brain_methods(query)

['mistakenness']

## Brain run selector

In [10]:
from links.brain_run_selector import select_brain_runs

In [1]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [12]:
query = "most unique images from the dataset with key 'uniqueness1 that are similar to image 10"
select_brain_runs(dataset, query, ["uniqueness", "image_similarity"])

{'uniqueness': 'uniqueness1', 'image_similarity': 'img_sim'}

In [16]:
query = "Five most mistaken images from the dataset with ground truth field 'predictions"
select_brain_runs(dataset, query, ["mistakenness"])

{'mistakenness': 'mistakenness'}

## Field selector

In [1]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [2]:
from links.field_selector import select_fields

In [8]:
query = "sort pred1 by number of detections"
fields = select_fields(dataset, query)
print(fields)

[id: string, filepath: string, tags: list, ground_truth: Detections, predictions: Detections, classif: Classification, mistakenness1_eval_fp: int, pred1: Detections]
[pred1]


## Label class selector

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

dataset = foz.load_zoo_dataset("quickstart")

Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [2]:
from links.label_class_selector import select_label_classes

In [8]:
query = "sort Bear and glove predictions by number of detections"
prompt = select_label_classes(dataset, query, ["predictions"])
print(prompt)

['Bear', 'glove']
['airplane', 'apple', 'backpack', 'banana', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tv', 'umbrella', 'vase', 'wine glass', 'zebra']
Matching Bear with bear
['airplane', 'apple', 'backpack', 'banana', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat',

In [4]:
sample = dataset.first()

In [49]:
print(sample)

<Sample: {
    'id': '642f8fb7986dad05794d2b9d',
    'media_type': 'image',
    'filepath': '/Users/jacobmarks/fiftyone/quickstart/data/000880.jpg',
    'tags': ['validation'],
    'metadata': None,
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '5f452471ef00e6374aac53c8',
                'attributes': {},
                'tags': [],
                'label': 'bird',
                'bounding_box': [0.21084375, 0.0034375, 0.46190625, 0.9442083333333334],
                'mask': None,
                'confidence': None,
                'index': None,
                'area': 73790.37944999996,
                'iscrowd': 0.0,
                'mistakenness': 0.01245725154876709,
                'mistakenness_loc': 0.2903442955979618,
                'mistakenness5': 0.01245725154876709,
                'mistakenness5_loc': 0.2903442955979618,
            }>,
            <Detection: {
                'id': '5f452471ef00e6374aac53c9'

In [45]:
def get_field_type(dataset, field_name):
    sample = dataset.first()
    field = sample.get_field(field_name)
    field_type = type(field).__name__
    return field_type

In [52]:
get_field_type(dataset, "mistakenness")

'float'

In [43]:
str(type(sample.get_field("predictions"))).split(".")[-1].split('\'')[0]

'Detections'

In [13]:
dataset.distinct("classif.label")

['alligator']

In [ ]:
# dataset.select_labels

In [11]:
dataset.distinct("predictions.detections.label")

['airplane',
 'apple',
 'backpack',
 'banana',
 'baseball glove',
 'bear',
 'bed',
 'bench',
 'bicycle',
 'bird',
 'boat',
 'book',
 'bottle',
 'bowl',
 'broccoli',
 'bus',
 'cake',
 'car',
 'carrot',
 'cat',
 'cell phone',
 'chair',
 'clock',
 'couch',
 'cow',
 'cup',
 'dining table',
 'dog',
 'donut',
 'elephant',
 'fire hydrant',
 'fork',
 'frisbee',
 'giraffe',
 'hair drier',
 'handbag',
 'horse',
 'hot dog',
 'keyboard',
 'kite',
 'knife',
 'laptop',
 'microwave',
 'motorcycle',
 'mouse',
 'orange',
 'oven',
 'parking meter',
 'person',
 'pizza',
 'potted plant',
 'refrigerator',
 'remote',
 'sandwich',
 'scissors',
 'sheep',
 'sink',
 'skateboard',
 'skis',
 'snowboard',
 'spoon',
 'sports ball',
 'stop sign',
 'suitcase',
 'surfboard',
 'teddy bear',
 'tennis racket',
 'tie',
 'toaster',
 'toilet',
 'toothbrush',
 'traffic light',
 'train',
 'truck',
 'tv',
 'umbrella',
 'vase',
 'wine glass',
 'zebra']

In [33]:
# sample = fo.Sample(filepath="/path/to/image.png")

# sample["ground_truth"] = fo.Classifications(
#     classifications=[
#         fo.Classification(label="animal"),
#         fo.Classification(label="cat"),
#         fo.Classification(label="tabby"),
#     ]
# )
# sample["prediction"] = fo.Classifications(
#     classifications=[
#         fo.Classification(label="animal", confidence=0.99),
#         fo.Classification(label="cat", confidence=0.98),
#         fo.Classification(label="tabby", confidence=0.72),
#     ]
# )

# print(sample)

sample = fo.Sample(filepath="/path/to/image.png")

# A simple polyline
polyline1 = fo.Polyline(
    label="circle",
    points=[[(0.3, 0.3), (0.7, 0.3), (0.7, 0.3)]],
    closed=False,
    filled=False,
)

# A closed, filled polygon with a label
polyline2 = fo.Polyline(
    label="triangle",
    points=[[(0.1, 0.1), (0.3, 0.1), (0.3, 0.3)]],
    closed=True,
    filled=True,
)

sample["polylines"] = fo.Polylines(polylines=[polyline1, polyline2])

sample["polyline"] = polyline2
sample["classif"] =fo.Classification(label="animal", confidence=0.99)

d = fo.Dataset()
d.add_sample(sample)



'64346d2a56f86d3fead8722d'

In [28]:
d.distinct("polylines.polylines.label")

['circle', 'triangle']

In [31]:
d.distinct("polyline.label")

['triangle']

In [34]:
d.distinct("classif.label")

['animal']

## DatasetView generator

In [1]:
from gpt_view_generator import get_gpt_view_text

In [2]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [3]:
query = "images with tables"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: images with tables
Identified likely view stages: ['match', 'sort_by_similarity', 'skip', 'limit', 'shuffle', 'take', 'exclude', 'exists', 'filter_labels', 'select_fields']
Identified brain methods: ['text_similarity']
Identified brain runs: {'text_similarity': 'clip'}
Identified potentially relevant fields: [ground_truth]
Class name table not found for label ground_truth
Identified label classes: {'ground_truth': ['table']}
[match(
    F("ground_truth.detections.label").contains("table")
)]


In [4]:
query = "high confidence detections"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: high confidence detections
Identified likely view stages: ['filter_labels', 'match', 'select_fields', 'exists', 'filter_field', 'set_field', 'match_labels', 'sort_by_similarity', 'exclude', 'exclude_by']
Identified potentially relevant fields: [predictions]
Identified label classes: {'predictions': []}
[filter_labels("predictions", F("confidence") > 0.95)]


In [5]:
query = "first 30 images with a ground truth detection"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: first 30 images with a ground truth detection
Identified likely view stages: ['match', 'sort_by_similarity', 'skip', 'filter_labels', 'limit', 'exclude', 'filter_field', 'limit_labels', 'select_fields', 'take']
Identified potentially relevant fields: [ground_truth]
Identified label classes: {'ground_truth': []}
[match(F("ground_truth.detections").length() > 0), limit(30)]


In [3]:
query = "sort by image uniqueness1 for images with a >0.5 confidence classification"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: sort by image uniqueness1 for images with a >0.5 confidence classification
Identified likely view stages: ['sort_by', 'sort_by_similarity', 'filter_labels', 'match', 'exists', 'limit', 'set_field', 'match_labels', 'select_fields', 'skip']
Identified brain methods: ['uniqueness']
Identified brain runs: {'uniqueness': 'uniqueness1'}
Identified potentially relevant fields: [classif, mistakenness1_eval_fp]
Identified label classes: {'classif': []}
[match(F("classif.confidence") > 0.5), sort_by("uniqueness1")]


In [4]:
query = "find the images most resembling a farm scene"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: find the images most resembling a farm scene
Identified likely view stages: ['match', 'sort_by_similarity', 'limit', 'geo_near', 'skip', 'exclude', 'exists', 'filter_labels', 'select_fields', 'sort_by']
Identified brain methods: ['image_similarity', 'text_similarity']
Identified brain runs: {'image_similarity': 'img_sim', 'text_similarity': 'img_sim'}
Identified potentially relevant fields: [classif]
Class name farm not found for label classif
Identified label classes: {'classif': ['farm']}
[sort_by_similarity("farm", k=15, brain_key="img_sim")]


## Roadmap

### Validation and error handling
- even fully fleshed out, this is not going to be 100% accurate. Need to validate that it is actually creating a valid DataSetView
- if it isn't, prompt the user for more specific information
    - this could potentially be tailored to which part of the process it failed

### Plugin
- Everything will be wrapped in a single python function `generate_datasetview_with_chatgpt(dataset, prompt)`
- Then need to turn this into a plugin. It will take the `session.dataset` as dataset, and will set `session.view`
- This will probably be a menu-item plugin - we could use the ChatGPT symbol - it will take user input
- Would love to have a toggle the user can specify for whether they want 
    - the view created from scratch, or 
    - the view stages concatenated with their existing view

### Future
- Memory/chat history
- More general question-answering:
    - First stage: determine whether the user is asking a question about the entire dataset, or individual samples
    - Use ChatGPT as a dispatcher, deciding what other models/processes to invoke.
        - if it is about aggregations, then decide what FiftyOne aggregation to perform, and interpret the results
        - if it is about a single image, employ BLIPv2 or equivalent...